In [1]:
import pandas as pd
import networkx as nx
import numpy as np
import itertools
import random
import argparse
from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2
from collections import defaultdict
import csv

In [2]:
df = pd.read_csv('C:/Users/Stevens/Desktop/BIA 674/Project/data.csv')

In [3]:
del df['Unnamed: 0']

In [4]:
df = df[df['travel_time_new'] > 0]

In [5]:
data = df.groupby(['pickup_block', 'dropoff_block', 'tod'], as_index = False)['travel_time_new', 'trip_distance'].mean()

In [6]:
G = nx.from_pandas_dataframe(data, 'dropoff_block', 'pickup_block', ['trip_distance', 'travel_time_new'])

In [7]:
JFK = 'POINT (-73.78172081065929 40.64255665977522)'
MSG = 'POINT (-73.99322124315619 40.75004063488754)'
ATL = 'POINT (-73.9768500517178 40.68451057260739)'
SIF = 'POINT (-74.07382887206769 40.64376379622597)'
BPM = 'POINT (-73.82501782103452 40.86752279510091)'
locations = [SIF, MSG, JFK, ATL, BPM]
location_name = ['SIF', 'MSG', 'JFK', 'ATL', 'BPM']

In [8]:
nx.dijkstra_path(G, source=MSG, target=ATL, weight = 'travel_time_new')

['POINT (-73.99322124315619 40.75004063488754)',
 'POINT (-73.88011440438557 40.88077956887605)',
 'POINT (-73.94950263647713 40.80990727518716)',
 'POINT (-73.94596570951198 40.68637498313604)',
 'POINT (-73.94084136069742 40.84193187853194)',
 'POINT (-73.9768500517178 40.68451057260739)']

In [10]:
edges = pd.DataFrame(list(itertools.permutations(locations, 2)))
edges_name = pd.DataFrame(list(itertools.permutations(location_name, 2)))
edge_list = pd.concat([edges_name, edges], axis=1)
edge_list.columns = ['from_name', 'to_name', 'from_point', 'to_point']

In [12]:
edge_list['weight'] = ''
for i in range(0, len(edge_list)):
    edge_list.loc[i, 'weight'] = nx.dijkstra_path_length(G, source=edge_list.loc[i, 'from_point'], target=edge_list.loc[i, 'to_point'], weight = 'travel_time_new')

In [13]:
edge_list.pivot(index='from_name', columns='to_name', values='weight')

to_name,ATL,BPM,JFK,MSG,SIF
from_name,,,,,
ATL,None,71.0435,53.9488,51.7792,54.3824
BPM,71.0435,None,59.2042,59.2918,52.5361
JFK,53.9488,59.2042,None,39.4821,53.4916
MSG,51.7792,59.2918,39.4821,None,56.0109
SIF,54.3824,52.5361,53.4916,56.0109,None


In [14]:
distances = np.array(edge_list.pivot(index='from_name', columns='to_name', values='weight').values.tolist())

In [15]:
distances[distances == None] = float(0)

In [16]:
class CreateDistanceCallback(object):
  """Create callback to calculate distances between points."""
  def __init__(self):
    """Array of distances between points."""
    self.matrix = distances

  def Distance(self, from_node, to_node):
    return int(self.matrix[from_node][to_node])

In [17]:
def main():
    city_names = ['SIF', 'MSG', 'JFK', 'ATL', 'BPM']
    tsp_size = len(city_names)
    num_routes = 1    # The number of routes, which is 1 in the TSP.
    # Nodes are indexed from 0 to tsp_size - 1. The depot is the starting node of the route.
    depot = 0
    # Create routing model
    if tsp_size > 0:
        routing = pywrapcp.RoutingModel(tsp_size, num_routes, depot)
        search_parameters = pywrapcp.RoutingModel.DefaultSearchParameters()

        # Create the distance callback, which takes two arguments (the from and to node indices)
        # and returns the distance between these nodes.
        dist_between_nodes = CreateDistanceCallback()
        dist_callback = dist_between_nodes.Distance
        routing.SetArcCostEvaluatorOfAllVehicles(dist_callback)

        # Solve, returns a solution if any.
        assignment = routing.SolveWithParameters(search_parameters)
        if assignment:
          # Solution cost.
          print("Total distance: " + str(assignment.ObjectiveValue()) + " minutes\n")
          # Inspect solution.
          # Only one route here; otherwise iterate from 0 to routing.vehicles() - 1
          route_number = 0
          index = routing.Start(route_number) # Index of the variable for the starting node.
          route = ''
          while not routing.IsEnd(index):
            # Convert variable indices to node indices in the displayed route.
            route += str(city_names[routing.IndexToNode(index)]) + ' -> '
            index = assignment.Value(routing.NextVar(index))
          route += str(city_names[routing.IndexToNode(index)])
          print("Route:\n\n" + route)
        else:
          print('No solution found.')
    else:
        print('Specify an instance greater than 0.')

if __name__ == '__main__':
  main()

Total distance: 255 minutes

Route:

SIF -> ATL -> JFK -> MSG -> BPM -> SIF


In [ ]:
# URL of Google maps to compare:
# https://www.google.com/maps/dir/40.643778,-74.073833/40.6845,+-73.9768611/40.8675278,+-73.8250278/40.6425567,+-73.7817208/40.750055599999996,+-73.99322219999999/3+Ferry+Terminal+Viaduct,+Staten+Island,+NY+10301/@40.7349703,-74.0742478,11z/data=!3m1!4b1!4m25!4m24!1m0!1m3!2m2!1d-73.9768611!2d40.6845!1m3!2m2!1d-73.8250278!2d40.8675278!1m3!2m2!1d-73.7817208!2d40.6425567!1m3!2m2!1d-73.9932222!2d40.7500556!1m5!1m1!1s0x89c24fd30eeee1db:0x702aa6f61e3ed0e9!2m2!1d-74.0745615!2d40.6427644!3e0

In [18]:
graph_data = data.copy()
graph_data['pickup_block'] = graph_data['pickup_block'].astype(str)
graph_data['dropoff_block'] = graph_data['dropoff_block'].astype(str)
graph_data = graph_data[['pickup_block', 'dropoff_block', 'travel_time_new']]

In [19]:
graph_data 

,pickup_block,dropoff_block,travel_time_new
0,POINT (-73.70950861306055 40.75286966142558),POINT (-73.94982363984116 40.82488938393518),16.550000
1,POINT (-73.70950861306055 40.75286966142558),POINT (-73.94982363984116 40.82488938393518),8.583333
2,POINT (-73.70950861306055 40.75286966142558),POINT (-73.94982363984116 40.82488938393518),14.600000
3,POINT (-73.71332340166876 40.7278398594051),POINT (-73.76903577175372 40.75418710120402),0.783333
4,POINT (-73.71332340166876 40.7278398594051),POINT (-73.85073541904542 40.89010821558506),7.500000
5,POINT (-73.72011603573252 40.74386571372121),POINT (-73.879524080934 40.67089676578712),2.866667
6,POINT (-73.72096515421497 40.72562044616777),POINT (-73.87858777184172 40.75735273845389),43.800000
7,POINT (-73.72307957769996 40.73516749289026),POINT (-73.81524254956227 40.6937058670494),13.283333
8,POINT (-73.72414725078478 40.74754070637903),POINT (-73.97890770261799 40.78636258552761),20.333333
9,POINT (-73.72414725078478 40.74754070637903),POINT (-73.97890770261799 40.78636258552761),26.466667
